In [7]:
!pip install -r requirements.txt 

  Using cached pandas_profiling-3.2.0-py2.py3-none-any.whl.metadata (21 kB)
INFO: pip is looking at multiple versions of pandas-profiling to determine which version is compatible with other requirements. This could take a while.
  Using cached pandas_profiling-3.1.0-py2.py3-none-any.whl.metadata (23 kB)
  Using cached pandas_profiling-3.0.0-py2.py3-none-any.whl.metadata (21 kB)
  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached visions-0.7.1-py3-none-any.whl.metadata (5.5 kB)
  Using cached htmlmin-0.1.12.tar.gz (19 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [27 lines of output]
      Traceback (most recent call last):
        File "/Users/leonardocipriani/Documents/dev/python/Artificial Intelligence Projects/oca-ia/.venv/lib/python3.13/site-packages/pip/_vendor/pyproject_hooks/_i

## Exemplo de classificador com leitura do CSV. 

Etapas:

- Leitura CSV.
- Tratamento de imagens com lib PIL. 
- Leitura imagem. 
- Aplicacao Filtro para remocao background


Input do modelo : (Imagem, sexo, idade, etnia)

In [8]:
import pandas as pd
from PIL import Image, ImageChops
import numpy as np
import torch
from lib.ImageFIlter import treat_image_PIL
from sklearn.model_selection import KFold
from torch.utils.data import  DataLoader, TensorDataset, Dataset
from torch import nn
import timm
from tqdm.notebook import tqdm
import torch.optim as optim
import os

In [9]:
## Geracao dataset com aleatorio
ds_file = pd.read_csv('dataset/exemplo_csv_2.csv')
arr = np.random.choice([0, 1], size=216)

ds_file = ds_file[['path','image_id','ID','IDADE','SEXO','ETNIA']]
ds_file['oca'] = arr
ds_file.to_csv('dataset/oca.csv')

In [10]:
ds_file = pd.read_csv('dataset/oca.csv')


## Geracao Tensor com imagens filtradas

In [11]:
## Loop
img_dataset = np.ones((ds_file.shape[0],3,256,256),dtype=np.uint8)

j=0
for i in ds_file['path']:
    img_dataset[j]=treat_image_PIL('dataset/path/'+i,2)
    j+=1
tensor_imagem = torch.tensor(img_dataset)



## Geracao Tensor Age, sex, Etinia

Encoding: 

- Sexo: M=0 F=1

- Idade: Normalizar? Sim, dividir por 100 igual o artigo do Marco e Felipe fizeram. 

- Etnia: Encoding simples de categorias. Utilizaremos o Label Encoding para isso 

In [12]:
ds_id_sex_et= ds_file[['IDADE', 'SEXO', 'ETNIA']]
ds_id_sex_et['SEXO'] = ds_file['SEXO'].map({ 'M':0, 'F':1})
ds_id_sex_et['IDADE'] = ds_file['IDADE']/100

/var/folders/30/pgp77_r92w5_rn1ghx4kn51h0000gn/T/ipykernel_52174/1074131203.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_id_sex_et['SEXO'] = ds_file['SEXO'].map({ 'M':0, 'F':1})
/var/folders/30/pgp77_r92w5_rn1ghx4kn51h0000gn/T/ipykernel_52174/1074131203.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_id_sex_et['IDADE'] = ds_file['IDADE']/100


In [13]:
## Encoding variavel ETNIA -> Usando label Encoding
from sklearn.preprocessing import LabelEncoder

# Create a sample dataframe with categorical data

#print(f"Before Encoding the Data:\n\n{ds_id_sex_et['ETNIA']}\n")

# Create a LabelEncoder object
le = LabelEncoder()

# Fit and transform the categorical data
ds_id_sex_et['ETNIA'] = le.fit_transform(ds_id_sex_et['ETNIA'])
ds_id_sex_et

/var/folders/30/pgp77_r92w5_rn1ghx4kn51h0000gn/T/ipykernel_52174/2772605160.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_id_sex_et['ETNIA'] = le.fit_transform(ds_id_sex_et['ETNIA'])


,IDADE,SEXO,ETNIA
0,0.85,0,1
1,0.32,1,1
2,0.60,1,1
3,0.57,1,4
4,0.57,1,1
...,...,...,...
211,0.35,1,3
212,0.72,0,2
213,0.71,1,2
214,0.49,0,2


## Geracao de rótulos para classificação

Colunas utilizadas para classificacao (Labels)

labels = [ OCA, NOCA]

In [14]:
##
ds_file.columns
labels = [ 'oca']
ds_labels = ds_file[labels]
ds_labels=ds_labels.astype(int)#,'False':0})
tensor_label = torch.tensor(np.array(ds_labels))
tensor_label

tensor([[1],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],

### Resumo:

model(tensor_imagem,ds_id_sex_et)

labels : tensor_label

## Criacao Subset 



In [15]:
class Subset(Dataset):
    r"""
    Subset of a dataset at specified indices.

    Arguments:
        dataset (Dataset): The whole Dataset
        indices (sequence): Indices in the whole set selected for subset
    """
    def __init__(self, dataset, indices):
        self.dataset = dataset
        self.indices = indices

    def __getitem__(self, idx):
        return self.dataset[self.indices[idx]]

    def __len__(self):
        return len(self.indices)

    @property
    def classes(self):
        return self.data.classes

## K-fold = 10


In [16]:
kf = KFold(n_splits=5)
kf.get_n_splits(tensor_imagem)
print(kf)
#for i in enumerate(kf.split(tensor_imagem)):
#    print(i)

KFold(n_splits=5, random_state=None, shuffle=False)


## Criacao do dataset de treino e validacao com batchsize

In [17]:
BATCH_SIZE = 16
train_dataset = TensorDataset(tensor_imagem, tensor_label)
train_loader_img = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [18]:
train_dataset[0][0]

tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8)

## Create a model to concat layers

O modelo utilizado será o mesmo do artigo: Artificial Intelligence-Driven Screening System for Rapid Image-Based Classification of 12-Lead ECG Exams: A Promising Solution for Emergency Room Prioritization 

Desenvolvido conforme a figura abaixo: 

![image.png](attachment:image.png)

In [19]:
#from models import ECGClassifierResnet

class ECGClassifierResnet(nn.Module):
    def __init__(self, num_classes=5):
        super(ECGClassifierResnet, self).__init__()
        # Where we define all the parts of the model
        #self.base_model = timm.create_model('efficientnet_b0', pretrained=True) 
        self.base_model=timm.create_model('resnet50d.ra4_e3600_r224_in1k',pretrained=True)
        #self.base_model = timm.create_model('vit_mediumd_patch16_reg4_gap_256.sbb2_e200_in12k_ft_in1k',num_classes=5,pretrained=True)


        self.features = nn.Sequential(*list(self.base_model.children())[:-1])

        enet_out_size = 2048        # Make a classifier
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(enet_out_size,1)
        ) # saida como Sigmoid multilabel

    def forward(self, x):
        # Connect these parts and return the output
        x = self.features(x)
        output = self.classifier(x)
        return output

## Train - Valid Loop



In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: " + str(device))
def simple_loop(model, train_image, val_image, epochs):
    # Simple training loop
    num_epochs = epochs
    train_losses, val_losses = [], []
    lim_loss = 1.5

    #model = modelo( num_classes=5)
    model.to(device)
    #criterion =  nn.BCEWithLogitsLoss()
    criterion =  nn.CrossEntropyLoss()
    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    predict_label_full = np.array([[1,1,1,1,1],[1,1,1,1,1]], dtype=int)
    true_label_full = np.array([[1,1,1,1,1],[1,1,1,1,1]], dtype=int)
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss_train = 0.0
        for images, labels in tqdm(train_image, desc='Training loop'):
            # Move inputs and labels to the device
            images = images.to(torch.float)
            image, label = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(image)
            loss_train = criterion(outputs.float(), label.float())
            loss_train.backward()
            optimizer.step()
            running_loss_train += loss_train.item() * label.size(0)
        train_loss = running_loss_train / len(train_image.dataset)
        train_losses.append(train_loss)
        model.eval()
        running_loss_valid = 0.0
        rotulos =[] 
        predict_label =np.array([[1,1,1,1,1],[1,1,1,1,1]], dtype=int)
        true_label =np.array([[1,1,1,1,1],[1,1,1,1,1]], dtype=int)
        with torch.no_grad():
            for images, labels in tqdm(val_image, desc='Validation loop'):
                # Move inputs and labels to the device
                images = images.to(torch.float)
                images, label = images.to(device), labels.to(device)
                rotulos.append(label.cpu().data.numpy())
                outputs = model(images)
                loss_valid = criterion(outputs.float(), label.float())
                #print( [outputs.cpu().data.numpy().astype(int).T[0]])
                #print(label.cpu().data.numpy().astype(int).T[0])
                #print(predict_label)
                try:
                    _pred = [outputs.cpu().data.numpy().astype(int).T[0]]
                    print(_pred)
                    predict_label=np.concatenate((predict_label,_pred), axis=0)
                    print(predict_label)
                    _true = [label.cpu().data.numpy().astype(int).T[0]]
                    true_label=np.concatenate((true_label, _true), axis=0)
                except Exception as e:
                    print(f"Concatenation error: {e}")
                    print(_pred)
                    print(predict_label)    
                running_loss_valid += loss_valid.item() * label.size(0)
        val_loss = running_loss_valid / len(val_image.dataset)
        val_losses.append(val_loss)
        try:
            predict_label_full=np.concatenate((predict_label_full,[predict_label]), axis=0)
            true_label_full=np.concatenate((true_label_full,[true_label]), axis=0)
        except Exception as e:
            print(f"Concatenation error: {e}")
            print(predict_label)
            print(true_label)
        #val_acc = accuracy_score(rotulos,output_model)
        print(f'Val accuracy {1}')
        print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {train_loss}, Validation loss: {val_loss}")
    
    return train_losses, val_losses, model,predict_label_full, true_label_full

Device: cpu


In [316]:
np.array([[1,1,1,1,1],[1,1,1,1,1]], dtype=int)

array([[1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1]])

In [317]:
a = np.array([[1,2,3,4,5],[1,2,3,4,5]], dtype=int)
b = np.array([6,7,8,9,10], dtype=int)
print(a)
c = np.concatenate((a,[b]), axis=0)
c

[[1 2 3 4 5]
 [1 2 3 4 5]]


array([[ 1,  2,  3,  4,  5],
       [ 1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10]])

In [318]:
predict_label =np.empty([2,2])
true_label =np.array([])
predict_label1 =np.empty([2,2], dtype=float)

a = np.zeros((2,2))
predict_label =np.append([predict_label, a])
predict_label

TypeError: append() missing 1 required positional argument: 'values'

In [319]:
print(np.ones([5], dtype=int))

[1 1 1 1 1]


In [320]:
def salvar_model(model, path = 'output/modelos', name_file='model.pth'):
    full_path = os.path.join(path, name_file)
    torch.save(model.state_dict(), full_path)

In [321]:
def salvar_metricas(path='output/metricas', name_file_train='train_loss_total.npy', name_file_val='val_loss_total.npy',predict_label=None, true_label=None):
    full_path_train = os.path.join(path, name_file_train)
    full_path_val = os.path.join(path, name_file_val)
    np.save(full_path_train, np.array(predict_label))
    np.save(full_path_val, np.array(true_label))

In [324]:

train_loss_total = []
val_loss_total =[]
all_models =[]
epochs = 1
## create first model.
for i, (train_index, test_index) in enumerate(kf.split(train_dataset)):
    print(f"Fold {i}:")
    #print(f"  Train: index={train_index}")
    #print(f"  Test:  index={test_index}")
    ## init train test for folder
    train_dataset_part = Subset( train_dataset, train_index)
    val_dataset_part = Subset( train_dataset, test_index)

    train_loader_img = DataLoader(train_dataset_part, batch_size=5, shuffle=True)
    val_loader_img = DataLoader(val_dataset_part, batch_size=5, shuffle=True)

    model= ECGClassifierResnet( num_classes=1)
    salvar_model(model, path='output/modelos', name_file=f'model_fold_{i}.pth')
    print(f'Train and valid for Fold {i}')
    t, l,_,outputs,labels = simple_loop(model, train_loader_img,val_loader_img, epochs)
    ## Evaluate model.
    salvar_metricas(path='output/metricas', name_file_train=f'predict_label_{i}.npy', name_file_val=f'true_label_{i}.npy', predict_label= outputs, true_label= labels)
    train_loss_total.append(t)
    val_loss_total.append(l)

Fold 0:
Train and valid for Fold 0


Training loop:   0%|          | 0/35 [00:00<?, ?it/s]

Validation loop:   0%|          | 0/9 [00:00<?, ?it/s]

Pred label
[array([0, 0, 0, 0, 0])]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 0 0 0]]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 1 0 1]]
Pred label
[array([0, 0, 0, 0, 0])]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 1 0 1]
 [1 0 1 0 0]]
Pred label
[array([0, 0, 0, 0, 0])]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 1 0 1]
 [1 0 1 0 0]
 [1 0 1 1 1]]
Pred label
[array([0, 0, 0, 0, 0])]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 1 0 1]
 [1 0 1 0 0]
 [1 0 1 1 1]
 [1 1 0 0 1]]
Pred label
[array([0, 0, 0, 0, 0])]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 1 0 1]
 [1 0 1 0 0]
 [1 0 1 1 1]
 [1 1 0 0 1]
 [0 0 1 1 0]]
Pred label
[array([0, 0, 0, 0, 0])]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[[1 1 1 1 1]
 [1 1 

Training loop:   0%|          | 0/35 [00:00<?, ?it/s]

...

Validation loop:   0%|          | 0/9 [00:00<?, ?it/s]

Pred label
[array([0, 0, 0, 0, 0])]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 0 0 0]]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 1 1 0 0]]
Pred label
[array([0, 0, 0, 0, 0])]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 1 1 0 0]
 [0 1 0 0 0]]
Pred label
[array([0, 0, 0, 0, 0])]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 1 1 0 0]
 [0 1 0 0 0]
 [0 0 1 1 0]]
Pred label
[array([0, 0, 0, 0, 0])]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 1 1 0 0]
 [0 1 0 0 0]
 [0 0 1 1 0]
 [0 0 1 0 1]]
Pred label
[array([0, 0, 0, 0, 0])]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 1 1 0 0]
 [0 1 0 0 0]
 [0 0 1 1 0]
 [0 0 1 0 1]
 [1 1 1 1 0]]
Pred label
[array([0, 0, 0, 0, 0])]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[[1 1 1 1 1]
 [1 1 

Training loop:   0%|          | 0/35 [00:00<?, ?it/s]

Validation loop:   0%|          | 0/9 [00:00<?, ?it/s]

Pred label
[array([0, 0, 0, 0, 0])]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 0 0 0]]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 1 1 1 0]]
Pred label
[array([0, 0, 0, 0, 0])]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 1 1 1 0]
 [0 1 0 1 1]]
Pred label
[array([0, 0, 0, 0, 0])]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 1 1 1 0]
 [0 1 0 1 1]
 [1 1 1 0 1]]
Pred label
[array([0, 0, 0, 0, 0])]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 1 1 1 0]
 [0 1 0 1 1]
 [1 1 1 0 1]
 [1 1 1 1 1]]
Pred label
[array([0, 0, 0, 0, 0])]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 1 1 1 0]
 [0 1 0 1 1]
 [1 1 1 0 1]
 [1 1 1 1 1]
 [0 1 1 1 0]]
Pred label
[array([0, 0, 0, 0, 0])]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[[1 1 1 1 1]
 [1 1 

In [301]:
outputs

array([    1,     1,     1,     1,     1,     1,     1,     1,     1,
           1, 22180, 21973, 22180, 17202, 29978, 22180, 22180, 22180,
       38713, 22571, 31117, 38375, 22180, 17712, 26842, 22180, 22180,
       22180, 22180, 22180, 26452, 22180, 22180, 22180, 22180, 22180,
       35583, 16483, 22180, 22180, 22180, 22180, 22180, 22180, 22180,
       22180, 22180, 22180, 29720, 22180, 22180, 32061, 26868, 22180])

## Métricas de Avaliacao dos modelos

Acuracia

Precisao

Sensibilidade

Especificidade

F1-Score

In [325]:
#### Leitura de métricas salvas
import numpy as np
predict_label_0 = np.load('output/metricas/predict_label_0.npy')
true_label_0 = np.load('output/metricas/true_label_0.npy')
# --- IGNORE ---


In [326]:
predict_label_0


array([[1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1]])

In [51]:
220/16

13.75

In [49]:
print("a")
print(type(predict_label_0))
predict_label_0.shape

a
<class 'numpy.ndarray'>


(220,)

## Criacao graficos de treinamento e validacao

In [ ]:
## Criar graficos de treinamento e validacao
import matplotlib.pyplot as plt
for i in range(5):
    plt.figure(figsize=(10,5))
    plt.plot(train_loss_total[i], label='Train Loss')
    plt.plot(val_loss_total[i], label='Validation Loss')
    plt.title(f'Fold {i} - Train and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(f'output/graficos/loss_fold_{i}.png')
    plt.show()
    plt.close()
